# Understanding TensorRT - Convert Graph

## Convert the Model
1. checkpoint -> Frozen TensorFlow graph  
2. Frozen TensorFlow graph -> TFLite  
3a. TFLite -> EdgeTPU (Google Coral)  
3b. TFLite -> TensorRT (NVIDIA GPU)  

### Did you train locally or via SageMaker Docker?
- Locally - the training checkpoint is in code/model
- SageMaker - the training checkpoint was pushed to S3 then pulled back down and extracted to /trained_model

The conversion script expects it to be in /trained_model


## CUDA / TensorFlow Versions
### TF Lite
No big deal and there are probably no concerns here

### TensorRT 7.0
TensorRT 7.0 says the model must be trained w/ TensorFlow 1.14 (which requires CUDA 10.0).  
- did you train w/ 1.14  
- did you convert to tflite w/ 1.14  

just beware

### Edge TPU (Google Coral)
Hmm, this all worked with TF 1.13 and 1.14, haven't tested 1.15

In [1]:
import os
import sys

In [2]:
import tensorflow as tf

print (tf.__version__)

1.14.1


/home/jayson/anaconda3/envs/tf114/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jayson/anaconda3/envs/tf114/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jayson/anaconda3/envs/tf114/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jayson/anaconda3/envs/

## GLOBALS

In [3]:
# GLOBALS so work with the bash scripts
PROJECT_DIR = os.getcwd()
TASKS_DIR = os.path.join(PROJECT_DIR, 'tasks')
CODE_DIR = os.path.join(PROJECT_DIR, 'code')
TFLITE_DIR = os.path.join(PROJECT_DIR, 'tflite_model')
TENSORFLOW_DIR = os.path.join(PROJECT_DIR, 'tensorflow_model')
MODEL_RESEARCH_DIR = os.path.join(PROJECT_DIR, 'code/models/research')
SLIM_DIR = os.path.join(MODEL_RESEARCH_DIR, 'slim')
TRAINED_DIR = os.path.join(PROJECT_DIR, 'trained_model')

NUM_TRAINING_STEPS = 6000
# Local (non-SageMaker) -or- SageMaker
# PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# PICK 1 - A. Local or  B. Hosted or C. Original 

## A - Locally Trained
### get the checkpoint files to the correct location

In [ ]:
# is your checkpoint (NUM_TRAINING_STEPS) there?
! ls {CODE_DIR}/model/*{NUM_TRAINING_STEPS}*

In [ ]:
# copy to the /trained_model directory
# this will make it compatible with the convert script
! cp {CODE_DIR}/model/*{NUM_TRAINING_STEPS}* {TRAINED_DIR}
! ls {TRAINED_DIR}/*{NUM_TRAINING_STEPS}* -l

## check the configuration

In [ ]:
# PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# verify
! ls {CODE_DIR}/{PIPELINE_CONFIG}

## B - Trained in SageMaker

In [ ]:
! ls {TRAINED_DIR}/*{NUM_TRAINING_STEPS}* -l

In [ ]:
PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
# PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# verify
! ls {CODE_DIR}/{PIPELINE_CONFIG}

## C - Convert Original MobileNet to tflite_graph.pb
The original MobileNet SSD model - that this project trained on top of - is stored in ssd-dag/code/ckpt.  

If you want to use this model (instead of detect Chick-fil-A products) you first need to convert it convert to a TensorFlow Frozen Graph or a TFLite graph.  Assuming you put the MobileNet in code/ckpt (per the instructions in TrainModel_Step1_Local) - this will set up the environment variables for you to do that very easily.

In [4]:
! cp {CODE_DIR}/ckpt/pipeline.config {CODE_DIR}/local_mobilenet_v1_ssd_coco.config
PIPELINE_CONFIG='local_mobilenet_v1_ssd_coco.config'

NUM_TRAINING_STEPS='0'
! cp {CODE_DIR}/ckpt/model.ckpt.data-00000-of-00001 {TRAINED_DIR}/model.ckpt-0.data-00000-of-00001
! cp {CODE_DIR}/ckpt/model.ckpt.index {TRAINED_DIR}/model.ckpt-0.index
! cp {CODE_DIR}/ckpt/model.ckpt.meta {TRAINED_DIR}/model.ckpt-0.meta

cp: cannot stat '/home/jayson/projects/ssd-dag/code/ckpt/pipeline.config': No such file or directory
cp: cannot stat '/home/jayson/projects/ssd-dag/code/ckpt/model.ckpt.data-00000-of-00001': No such file or directory
cp: cannot stat '/home/jayson/projects/ssd-dag/code/ckpt/model.ckpt.index': No such file or directory
cp: cannot stat '/home/jayson/projects/ssd-dag/code/ckpt/model.ckpt.meta': No such file or directory


# Convert the Checkpoint to Frozen Graph & TF Lite Graph

In [6]:
# convert checkpoint is a task script - located in the tasks/ directory
os.chdir(TASKS_DIR)  
! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num {NUM_TRAINING_STEPS} --pipeline_config {PIPELINE_CONFIG}


TASKS_DIR=/home/jay/projects/ssd-dag/tasks
***
/home/jay/projects/ssd-dag/tflite_model
:/home/jay/projects/ssd-dag/code/models/research/slim:/home/jay/projects/ssd-dag/code/models/research
+ ckpt_number=0
+ [[ 4 -gt 0 ]]
+ case "$1" in
+ ckpt_number=0
+ shift 2
+ [[ 2 -gt 0 ]]
+ case "$1" in
+ pipeline_config=local_mobilenet_v1_ssd_coco.config
+ shift 2
+ [[ 0 -gt 0 ]]
+ rm /home/jay/projects/ssd-dag/tensorflow_model -rf
+ rm /home/jay/projects/ssd-dag/tflite_model -rf
+ echo '-- check for model checkpoint (the raw graph):,' , 0
-- check for model checkpoint (the raw graph):, , 0
+ ls /home/jay/projects/ssd-dag/trained_model/eval_0 /home/jay/projects/ssd-dag/trained_model/events.out.tfevents.1564578737.ip-10-0-230-217.ec2.internal /home/jay/projects/ssd-dag/trained_model/model.ckpt-0.data-00000-of-00001 /home/jay/projects/ssd-dag/trained_model/model.ckpt-0.index /home/jay/projects/ssd-dag/trained_model/model.ckpt-0.meta /home/jay/projects/ssd-dag/trained_model/model.ckpt-502.data-00000


W0131 14:49:52.473802 140535112308544 deprecation_wrapper.py:119] From /home/jay/projects/ssd-dag/code/models/research/object_detection/core/preprocessor.py:2660: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.


W0131 14:49:52.502225 140535112308544 deprecation_wrapper.py:119] From /home/jay/projects/ssd-dag/code/models/research/object_detection/meta_architectures/ssd_meta_arch.py:575: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0131 14:49:52.569839 140535112308544 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7fd0827b7b70>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7

W0131 14:49:52.931259 140535112308544 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7fd08268a358>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7fd08268a358>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0131 14:49:52.990343 140535112308544 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7fd08262aa90>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: conv

W0131 14:49:53.270966 140535112308544 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7fd082768080>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7fd082768080>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0131 14:49:53.306332 140535112308544 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7fd082617080>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach th

W0131 14:49:53.617934 140535112308544 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7fd0822b61d0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7fd0822b61d0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0131 14:49:53.647956 140535112308544 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7fd0822854e0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Batch

...

W0131 14:50:16.718518 139682505467712 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09ffb50f98>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09ffb50f98>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0131 14:50:16.859054 139682505467712 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09ffb50f98>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python

W0131 14:50:17.831105 139682505467712 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09ff2db7f0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09ff2db7f0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0131 14:50:17.901929 139682505467712 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09ff2db7f0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python

W0131 14:50:18.373003 139682505467712 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09fe997cc0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09fe997cc0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0131 14:50:18.440976 139682505467712 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f09fe27ec50>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python

Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0131 14:50:19.920467 139682505467712 deprecation.py:323] From /home/jay/anaconda3/envs/tensorRT/lib/python3.6/site-packages/tensorflow/python/tools/freeze_graph.py:127: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2020-01-31 14:50:20.423282: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1181] Device interconnect StreamExecutor with strength 1 edge matrix:
2020-01-31 14:50:20.423306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1187]      
INFO:tensorflow:Restoring parameters from /home/jay/projects/ssd-dag/trained_model/model.ckpt-0
I0131 14:50:20.424429 139682505467712 saver.py:1280] Restoring parameters from /home/jay/projects/ssd-dag/trained_model/model.ckpt-0
Instructions for updating:
Use `tf.compat.v1.graph_util.co

2020-01-31 14:50:24.210670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-01-31 14:50:24.211033: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5576ad34ad40 executing computations on platform CUDA. Devices:
2020-01-31 14:50:24.211068: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): GeForce GTX 1060 with Max-Q Design, Compute Capability 6.1
2020-01-31 14:50:24.234346: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2020-01-31 14:50:24.235099: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5576ad362020 executing computations on platform Host. Devices:
2020-01-31 14:50:24.235146: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2020-01-31 14:50:24.235256: I tensorflow/core/common_run

## Saved (output) Graphs

These models will now behave differently.   The API is different and the performance is different.   Furthermore the dependencies are different.  The JSON that is returned will not be compatible.   So, make sure you know what your doing.

### TensorFlow - Frozen Graph
Look for examples of using a Tensor frozen graph.  Look at notebook:  DetectModel_Step1_Local_TensorFlowFrozenGraph

### TensorFlow - TFLite
Look for examples of using TensorFlow Lite.   Look at notebook: DetectMOdel_Step1_Local_TFLite



In [ ]:
! ls {TENSORFLOW_DIR} -l

In [ ]:
! ls {TFLITE_DIR} -l